In [3]:
Blockchain = list()

In [ ]:
create_genesis_block(123)
for i in range(124,127):
    next_block(i, Blockchain[-1])
for blocks in Blockchain:
    print(blocks.data)
    print(blocks.previous_hash)

### Socket Programming-Threads

In [1]:
import threading
import socket
import pickle
import hashlib
from ecdsa import SigningKey

In [4]:
class Transaction:
    
    def __init__(self, fromm, to, amount, signature):
        self.fromm = fromm
        self.to = to
        self.amount = amount
        self.signature = signature
    
    def get_string(self):
        trans_string_obj = str(self.fromm)+str(self.to)+str(self.amount)
        return trans_string_obj

In [5]:
class Block:
    
    
    def __init__(self):
        self.block_id = 0
        self.nounce = 0
        self.transaction = list()
        self.previous_hash = None
    
    def get_current_hash(self):
        hash_val = hashlib.sha256((self.get_transaction_string_obj()).encode('utf-8'))
        return hash_val
    
    def get_transaction_string_obj(self):
        trans_string = str()
        for transaction in self.transaction:
            trans_string = trans_string + transaction.get_string()
        trans_string = trans_string + str(self.balance) + str(self.nounce) \
            + str(self.previous_hash)
        return trans_string
    

In [ ]:
def create_genesis_block(data):
    hash_val = hashlib.sha256(bytes(data))
    blk = Block(data=data, previous_hash=hash_val.hexdigest(), time=datetime.timestamp(datetime.now()))
    Blockchain.append(blk)

In [ ]:
def next_block(data, previous_block):
    hash_val = previous_block.get_current_hash(data)
    blk = Block(data=data, previous_hash=hash_val.hexdigest(), time=datetime.timestamp(datetime.now()))
    Blockchain.append(blk)

In [6]:
# client side

class Peer:
    
    
    def __init__(self, ip):
        self.MANAGER_IP = '192.168.1.109'
        self.MANAGER_PORT = 8800
        self.server_port = 5500
        self.client_ip = ip
        self.client_socket = socket.socket()
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM, 0)
        self.blockchain = list()
        self.server_thread = threading.Thread(target=self.start_server)
        self.server_thread.daemon = True
        self.server_thread.start()
        self.stop_server = False
        self.peers = list()
        self.balance = 0
        self.secret_key = SigningKey.get_verifying_key()
       
    
    def start_server(self):
        self.server_socket.bind((self.client_ip, self.server_port))
        self.server_socket.listen(10)
        while True:
            client_peer, address = self.server_socket.accept()
            # actions
            data = client_peer.recv(4096).decode('utf-8')
            # add  
            if data.startswith('add-peer'):
                self.peers.add(data.split()[1])
            elif data.startswith('remove-peer'):
                self.peers.remove(data.split()[1])
            else:
                transaction_obj = pickle.loads(client_peer.recv(4096))
                new_block = Block()
        self.server_socket.close()
      
    
    def _stop_server(self):
        self.stop_server = True
      
    
    def _inform_manager_add(self, message):
        sock = socket.socket()
        sock.connect((self.MANAGER_IP, self.MANAGER_PORT))
        sock.send(message.encode('utf-8'))
        peers = pickle.loads(sock.recv(4096))
        self.peers = peers
    
    
    def _inform_manager_delete(self, message):
        sock = socket.socket()
        sock.connect((self.MANAGER_IP, self.MANAGER_PORT))
        sock.send(message.encode('utf-8'))
    
    
    def leave_network(self):
        return self._inform_manager_delete(f"remove {self.client_ip}")
        
        
    def join_network(self):
        return self._inform_manager_add(f'add {self.client_ip}')
    
    
    def update_balance(self, amount):
        self.balance = amount
    
    def do_transaction(self, fromm, to, amount):
        if amount > self.balance:
            print("Not allowed")
            #actions
        else :
            message = str(fromm) + str(to) + str(amount)
            public_key = SigningKey.generate() # uses NIST192p
            secret_key = public_key.get_verifying_key()
            signature = public_key.sign(message.encode('utf-8'))
            transaction_obj = Transaction(fromm, to, amount, signature)
            for peer in self.peers:
                sock = socket.socket()
                sock.connect((peer, self.server_port))
                sock.send(pickle.dumps(transaction_obj))
                sock.close()

In [41]:
client = Peer('192.168.1.110')

In [42]:
client.join_network()

In [43]:
client.leave_network()

In [21]:
client._stop_server()

In [44]:
client.peers

set()

In [49]:
from ecdsa import SigningKey
sk = SigningKey.generate() # uses NIST192p
vk = sk.get_verifying_key()
signature = sk.sign("message".encode('utf-8'))
assert vk.verify(signature, "message".encode('utf-8'))